# csv转geopython
#https://www.it1352.com/1505417.html
#http://c.biancheng.net/view/5693.html(相对路径、绝对路径)

In [2]:
import csv, json
import pandas as pd
import numpy as np
from geojson import Feature, FeatureCollection, Polygon

features = []
data = pd.read_csv('../data/gxdc_tcd.csv', header = None)
print(len(data))

for i in range(1,len(data)):
#     print(data[0][i])
    lon1,lat1,lon2,lat2,lon3,lat3,lon4,lat4,lon5,lat5 = map(float, (data[1][i], data[2][i], data[3][i], data[4][i] , data[5][i], data[6][i], data[7][i], data[8][i], data[9][i], data[10][i]))
#     polygon = np.array([lon1,lat1],[lon2,lat2],[lon3,lat3],[lon4,lat4],[lon5,lat5])
    features.append(
        Feature(
            geometry = Polygon([[[lon1,lat1],[lon2,lat2],[lon3,lat3],[lon4,lat4],[lon5,lat5]]]),
            properties = {
                'FID': int(i),
                'ID': data[0][i]
            }
        )
    )
    
collection = FeatureCollection(features)
with open("../data/GeoObs.json", "w") as f:
    f.write('%s' % collection)

14072


# 订单数据处理

In [2]:
import pandas as pd

dingdan = pd.read_csv('../data/gxdc_dd_o.csv')

def data_process_wyc(data):
    data['UPDATE_TIME'] = pd.to_datetime(data['UPDATE_TIME'])
    # 将数据排序,并把排序后的数据赋值给原来的数据
    data = data.sort_values(by = ['BICYCLE_ID','UPDATE_TIME'])
    
    #让这几个字段的下一条数据赋值给新的字段，在字段名加个1，代表后面一条数据的值    
    
    data.loc[:,'BICYCLE_ID1'] = data['BICYCLE_ID'].shift(-1)
    data.loc[:,'LOCK_STATUS1'] = data['LOCK_STATUS'].shift(-1)
    data.loc[:,'LATITUDE1'] = data['LATITUDE'].shift(-1)
    data.loc[:,'LONGITUDE1'] = data['LONGITUDE'].shift(-1)
    data.loc[:,'UPDATE_TIME1'] = data['UPDATE_TIME'].shift(-1)
    
    print(data.head(5))
    return data 
dingdan = data_process_wyc(dingdan)
dingdan.to_csv('../data/共享单车订单gxdc_dd.csv', index = None)

                             BICYCLE_ID   LATITUDE   LONGITUDE  LOCK_STATUS  \
18748  0000ff105fd5f9099b866bccd157dc50  24.496578  118.142543            1   
30333  0000ff105fd5f9099b866bccd157dc50  24.481350  118.167496            1   
56536  0000ff105fd5f9099b866bccd157dc50  24.494801  118.147564            1   
45519  0000ff105fd5f9099b866bccd157dc50  24.491365  118.149005            1   
75690  0000ff105fd5f9099b866bccd157dc50  24.493236  118.141339            1   

              UPDATE_TIME                       BICYCLE_ID1  LOCK_STATUS1  \
18748 2020-12-21 06:50:00  0000ff105fd5f9099b866bccd157dc50           1.0   
30333 2020-12-21 08:44:00  0000ff105fd5f9099b866bccd157dc50           1.0   
56536 2020-12-22 08:14:00  0000ff105fd5f9099b866bccd157dc50           1.0   
45519 2020-12-22 08:23:00  0000ff105fd5f9099b866bccd157dc50           1.0   
75690 2020-12-22 08:54:00  0000ff105fd5f9099b866bccd157dc50           1.0   

       LATITUDE1  LONGITUDE1        UPDATE_TIME1  
18748  24.4

# 判断条件，生成共享自行车OD数据

基于完整的共享车租借OD记录进行电子停车位的绑定，原始数据中存在很多开锁不成功的记录对后续操作产生干扰，因此这里进行预处理

In [3]:
#判断条件：
dingdan = pd.read_csv('../data/共享单车订单gxdc_dd.csv')
#1.这条数据和下一条数据的车辆ID必须是相同的
dingdan.loc[:,'IF_STATUS'] = dingdan['LOCK_STATUS1']-dingdan['LOCK_STATUS']
dingdan = dingdan[(dingdan['IF_STATUS'] == 1)]
dingdan = dingdan[(dingdan['BICYCLE_ID1'] == dingdan['BICYCLE_ID'])]
dingdan.to_csv('../data/共享单车gxdc_od.csv', index = None)

# 数据表连接

In [5]:
'''
import pandas as pd
import numpy as np

left = pd.read_csv('../data/hub_distance_origin1.csv', encoding = 'gbk')
right = pd.read_csv('../data/gxdc_tcdindex.csv', encoding = 'gbk')
left = left.set_index('HubFENCEID')
right = right.set_index('HubFENCEID')
merge = left.join(right,on='HubFENCEID', how = 'outer', lsuffix='_l', rsuffix='_r')
merge.head(5)
merge.to_csv('../data/gxdc_tcdtest.csv', encoding = 'gbk')
'''

import pandas as pd
import numpy as np

left = pd.read_csv('../data/gxdc_result2-1.csv', encoding = 'gbk')
right = pd.read_csv('../data/../data/gxdc_tcdcount1.csv')
left = left.set_index('FID_1')
right = right.set_index('HubName')
merge = left.join(right,on='FID_1', how = 'outer', lsuffix='_l', rsuffix='_r')
merge.head(5)
merge.to_csv('../data/gxdc_result2-1.csv', encoding = 'gbk')

,OBJECTID,FENCE_ID,BELONG_ARE,Shape_Leng,Shape_Area,Hubcount0,Hubcount1,in-out,in-out/area,in/area,FID,BICYCLE_ID,LOCK_STATU,LATITUDE1,LONGITUDE1,UPDATE_TIM,HubDist
FID_1,,,,,,,,,,,,,,,,,
1,1,长乐路0_L_A17001,长乐路0,13.437863,7.964842,NaN,NaN,0,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,长乐路0_L_A17002,长乐路0,13.408450,7.774982,3.0,NaN,-3,-0.385853,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,长乐路0_L_A17003,长乐路0,13.786735,8.980655,NaN,NaN,0,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,长乐路0_L_A17004,长乐路0,13.460416,8.696193,6.0,2.0,-4,-0.459971,0.229986,2.0,2.0,2.0,2.0,2.0,2.0,2.0
5,5,长乐路0_L_A17005,长乐路0,16.023975,10.931636,8.0,1.0,-7,-0.640343,0.091478,1.0,1.0,1.0,1.0,1.0,1.0,1.0


# 统计每个停车位内的停车数量

In [6]:
import pandas as pd
tcd = pd.read_csv('../data/hubdistance_od0-1.csv', encoding = 'gbk')
# tcd['UPDATE_TIME'] = pd.to_datetime(tcd['UPDATE_TIME'])
# tcd = tcd[tcd['UPDATE_TIME'].dt.hour.isin(np.arange(7, 9))]
# print(len(tcd))
count = tcd.groupby('HubName').count()
count.to_csv('../data/gxdc_tcdcount0.csv')

# 统计每个自定义区域内的停车位数量

In [7]:
tcd = pd.read_csv('../data/gxdc_tcdindex.csv', encoding = 'gbk')
count = tcd.groupby('BELONG_ARE').count()
count.to_csv('../data/gxdc_areacount.csv', encoding = 'gbk')

# 统计每个自定义区域内的停车数量

In [ ]:

tcd = pd.read_csv('../data/result2-1/gxdc_result2-1.csv', encoding = 'gbk')
count = tcd.groupby(['BELONG_ARE'])['in/area'].sum().reset_index()

count.to_csv('../data/result2-1/result1.csv', encoding = 'gbk')

In [10]:
import pandas as pd

test = pd.read_csv('../data/result2-1/gxdc_result2-1.csv', encoding = 'gbk')
test.head(5)
data = test[(test['percentinout'] > 3.5)]
unique = data['BELONG_ARE'].unique()
len(unique)

222

In [37]:
unique = pd.DataFrame(unique)
unique.to_csv('../data/result2-1/area2-2.csv', encoding = 'gbk')

# 根据订单数据计算骑行时间

这里考虑时间问题是基于平时租借共享自行车会遇到开锁后发现自行车无法骑行，原地关闭的情况，
所以做了时间统计，但其实对于区域潮汐统计骑行时间一长的判断意义不大

In [11]:
import pandas as pd 
import datetime

test = pd.read_csv('../data/共享单车gxdc_od.csv', encoding = 'gbk')
test['UPDATE_TIME'] = pd.to_datetime(test['UPDATE_TIME'])
test['UPDATE_TIME1'] = pd.to_datetime(test['UPDATE_TIME1'])
# 计算时间差，添加round表示四舍五入
# test['bikingtime'] = round((test['UPDATE_TIME1'] - test['UPDATE_TIME']).dt.total_seconds() / (Time_Bandwith * 60))
test['bikingtime'] = (test['UPDATE_TIME1'] - test['UPDATE_TIME']).dt.total_seconds() / 60

In [5]:
test.to_csv('../data/共享单车gxdc_od.csv', encoding = 'gbk', index = None)

# 利用geohash对订单数据进行停车点的匹配

In [ ]:
#共享单车停车点位（电子围栏）数据为规范共享单车停车秩序，统一划定的共享单车停放区域。
# 读取数据
def bike_fence_format(s):
    s = s.replace('[', '').replace(']', '').split(',')
    s = np.array(s).astype(float).reshape(5, -1)
    return s

# 共享单车停车点位（电子围栏）数据
bike_fence = pd.read_csv(PATH + 'gxdc_tcd_0.csv')
bike_fence['FENCE_LOC'] = bike_fence['FENCE_LOC'].apply(bike_fence_format)

# 得出停车点 LATITUDE 范围
bike_fence['MIN_LATITUDE'] = bike_fence['FENCE_LOC'].apply(lambda x: np.min(x[:, 1]))
bike_fence['MAX_LATITUDE'] = bike_fence['FENCE_LOC'].apply(lambda x: np.max(x[:, 1]))

# 得到停车点 LONGITUDE 范围
bike_fence['MIN_LONGITUDE'] = bike_fence['FENCE_LOC'].apply(lambda x: np.min(x[:, 0]))
bike_fence['MAX_LONGITUDE'] = bike_fence['FENCE_LOC'].apply(lambda x: np.max(x[:, 0]))

from geopy.distance import geodesic
# 根据停车点 范围 计算具体的面积
bike_fence['FENCE_AREA'] = bike_fence.apply(lambda x: geodesic(
    (x['MIN_LATITUDE'], x['MIN_LONGITUDE']), (x['MAX_LATITUDE'], x['MAX_LONGITUDE'])
).meters, axis=1)

# 根据停车点 计算中心经纬度
bike_fence['FENCE_CENTER'] = bike_fence['FENCE_LOC'].apply(
    lambda x: np.mean(x[:-1, ::-1], 0)
)

In [ ]:
# 共享单车订单数据
bike_order = pd.read_csv(PATH + '共享单车订单gxdc_dd.csv')
bike_order = bike_order.sort_values(['BICYCLE_ID', 'UPDATE_TIME'])

# geohash经纬度匹配
bike_order['geohash'] = bike_order.apply(
    lambda x: geohash.encode(x['LATITUDE'], x['LONGITUDE'], precision=6), 
axis=1)

bike_fence['geohash'] = bike_fence['FENCE_CENTER'].apply(
    lambda x: geohash.encode(x[0], x[1], precision=6)
)

# 区域流量统计

In [ ]:
bike_order['UPDATE_TIME'] = pd.to_datetime(bike_order['UPDATE_TIME'])
bike_order['DAY'] = bike_order['UPDATE_TIME'].dt.day.astype(object)
bike_order['DAY'] = bike_order['DAY'].apply(str)

bike_order['HOUR'] = bike_order['UPDATE_TIME'].dt.hour.astype(object)
bike_order['HOUR'] = bike_order['HOUR'].apply(str)
bike_order['HOUR'] = bike_order['HOUR'].str.pad(width=2,side='left',fillchar='0')

# 日期和时间进行拼接
bike_order['DAY_HOUR'] = bike_order['DAY'] + bike_order['HOUR']

# 使用透视表统计每个区域在不同时间的入流量和出流量：

In [ ]:
bike_inflow = pd.pivot_table(bike_order[bike_order['LOCK_STATUS'] == 1], 
                   values='LOCK_STATUS', index=['geohash'],
                    columns=['DAY_HOUR'], aggfunc='count', fill_value=0
)

bike_outflow = pd.pivot_table(bike_order[bike_order['LOCK_STATUS'] == 0], 
                   values='LOCK_STATUS', index=['geohash'],
                    columns=['DAY_HOUR'], aggfunc='count', fill_value=0
)

In [ ]:
根据入流量和出流量，可以计算得到每个位置的留存流量，即(入流量-出流量)

In [ ]:
bike_remain = (bike_inflow - bike_outflow).fillna(0)

# 存在骑走的车数量 大于 进来的车数量
bike_remain[bike_remain < 0] = 0  

# 按照天求平均
bike_remain = bike_remain.sum(1)

## 这里先提取产生潮汐现象的停车位，根据共享自行车的占地面积情况，一般长度达1米左右，宽度应该在0.2-0.3米左右，所以1平米的正方形电子停车位大概能停3-4辆车，但是电子停车位的形状只有少量为接近正方形的形状，长方形的例如0.5m*2m的电子停车位则大概可以停4辆车，所以当留存车辆数/停车位面积之比大于4的时候，认为停车情况超出停车位的承载能力

In [ ]:
# 留存车辆 / 街道停车位总面积，计算得到密度
bike_density = bike_fence.groupby(['STREET'])['geohash'].unique().apply(
    lambda hs: np.sum([bike_remain[x] for x in hs])
) / bike_fence.groupby(['STREET'])['FENCE_AREA'].sum()

# 按照密度倒序
bike_density = bike_density.sort_values(ascending=False).reset_index()

## 然后提取bike_density大于4.0的电子停车位标注FENCE_TYPE为1，其余标注为0，进行测试后得分为18.7759